# 04: PostgreSQL/Supabase Integration for Text-to-SQL

Welcome to the PostgreSQL integration notebook! This notebook demonstrates how to connect LlamaIndex text-to-SQL to production PostgreSQL databases (including Supabase).

## Learning Objectives

By the end of this notebook, you will be able to:
- Connect LlamaIndex to PostgreSQL/Supabase databases
- Implement secure connection patterns
- Query real data with natural language

## Prerequisites

✅ **You should have:**
- Supabase account (or PostgreSQL server)
- Database credentials in `.env` file
- Sample data already inserted (customers, products, orders)
- OpenAI API key configured

**Note:** If you followed the setup instructions, you should have 500 rows of data already in your Supabase database!

## Why PostgreSQL/Supabase?

**PostgreSQL** is ideal for production text-to-SQL deployments:
- Industry-standard production database
- Robust security and role management
- Excellent performance for complex queries
- **Supabase** provides hosted PostgreSQL with great developer experience

---

**Ready to query your data with natural language!** 🚀

## Section 1: PostgreSQL Connection Setup

### 1.1 Import Required Libraries

In [1]:
# Standard library imports
import os
from typing import Optional
import logging

# Third-party imports
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from sqlalchemy.pool import QueuePool
import pandas as pd

# LlamaIndex imports
from llama_index.core import SQLDatabase
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.indices.struct_store import SQLTableRetrieverQueryEngine
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✓ All libraries imported successfully")

✓ All libraries imported successfully


### 1.2 Load Environment Variables

In [2]:
# Load environment variables
load_dotenv()

# OpenAI API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in .env file")

# PostgreSQL credentials
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
POSTGRES_HOST = os.getenv("POSTGRES_HOST", "localhost")
POSTGRES_PORT = os.getenv("POSTGRES_PORT", "5432")
POSTGRES_DB = os.getenv("POSTGRES_DB")

# Validate PostgreSQL credentials
if not all([POSTGRES_USER, POSTGRES_PASSWORD, POSTGRES_DB]):
    print("⚠️ PostgreSQL credentials not fully configured")
    print("Please set POSTGRES_USER, POSTGRES_PASSWORD, and POSTGRES_DB in your .env file")
    print("See .env.example for template")
else:
    print("✓ Environment variables loaded")
    print(f"  PostgreSQL Host: {POSTGRES_HOST}:{POSTGRES_PORT}")
    print(f"  Database: {POSTGRES_DB}")
    print(f"  User: {POSTGRES_USER}")

✓ Environment variables loaded
  PostgreSQL Host: db.vdbpeetljcwgoroemlxb.supabase.co:6543
  Database: postgres
  User: postgres


### 1.3 Create Secure PostgreSQL Connection

**Security Best Practices:**
- Use environment variables for credentials
- Enable connection pooling
- Use `pool_pre_ping` to verify connections
- Set appropriate pool sizes
- Use SSL when possible (add `?sslmode=require` to connection string)

In [3]:
def create_postgres_engine(
    user: str,
    password: str,
    host: str,
    port: str,
    database: str,
    pool_size: int = 5,
    max_overflow: int = 10,
    echo: bool = False,
    use_ssl: bool = False
):
    """Create PostgreSQL engine with connection pooling."""
    from urllib.parse import quote_plus
    
    # URL-encode credentials to handle special characters
    encoded_user = quote_plus(user)
    encoded_password = quote_plus(password)
    
    # Build connection string with encoded credentials
    connection_string = (
        f"postgresql+psycopg2://{encoded_user}:{encoded_password}"
        f"@{host}:{port}/{database}"
    )
    
    # Add SSL if requested
    if use_ssl:
        connection_string += "?sslmode=require"
    
    # Create engine with connection pooling
    engine = create_engine(
        connection_string,
        poolclass=QueuePool,
        pool_size=pool_size,  # Number of connections to keep open
        max_overflow=max_overflow,  # Max extra connections when pool is full
        pool_pre_ping=True,  # Verify connections before using
        echo=echo,  # Log SQL statements
        pool_recycle=3600,  # Recycle connections after 1 hour
    )
    
    return engine

# Example usage (replace with your credentials)
if all([POSTGRES_USER, POSTGRES_PASSWORD, POSTGRES_DB]):
    try:
        engine = create_postgres_engine(
            user=POSTGRES_USER,
            password=POSTGRES_PASSWORD,
            host=POSTGRES_HOST,
            port=POSTGRES_PORT,
            database=POSTGRES_DB,
            echo=False,  # Set to True to see SQL logs
            use_ssl=True  # Enable SSL for Supabase/remote databases
        )
        
        # Test connection
        with engine.connect() as conn:
            result = conn.execute(text("SELECT version()"))
            version = result.fetchone()[0]
            print(f"\n✓ Connected to PostgreSQL")
            print(f"  Version: {version[:50]}...")
            
    except Exception as e:
        print(f"\n✗ Connection failed: {e}")
        print("Please check your PostgreSQL credentials and server status")
else:
    print("\n⚠️ Skipping connection test - credentials not configured")
    print("This is a demonstration notebook. Configure .env to test with real database.")


✓ Connected to PostgreSQL
  Version: PostgreSQL 17.6 on aarch64-unknown-linux-gnu, comp...


## Section 2: Schema Discovery and Inspection

### 2.1 Inspect PostgreSQL Schema

In [4]:
# Create SQLDatabase object with your Supabase tables
sql_database = SQLDatabase(
    engine,
    schema="public",  # Supabase uses 'public' schema
    include_tables=["customers", "products", "orders"]  # The tables we created
)

# Get available tables
tables = sql_database.get_usable_table_names()
print(f"✓ Found {len(tables)} tables: {tables}")

# Inspect each table
print("\n" + "="*60)
print("TABLE SCHEMAS")
print("="*60)

for table_name in tables:
    print(f"\n{table_name.upper()}:")
    print(sql_database.get_single_table_info(table_name))
    
print("\n" + "="*60)
print("✓ Schema inspection complete!")
print("You now have 500 customers, 500 products, and 500 orders")
print("="*60)

✓ Found 3 tables: ['customers', 'orders', 'products']

TABLE SCHEMAS

CUSTOMERS:
Table 'customers' has columns: id (INTEGER), name (VARCHAR(100)), email (VARCHAR(100)), city (VARCHAR(50)), created_at (TIMESTAMP), .

ORDERS:
Table 'orders' has columns: id (INTEGER), customer_id (INTEGER), product_id (INTEGER), quantity (INTEGER), total_amount (NUMERIC(10, 2)), order_date (TIMESTAMP),  and foreign keys: ['customer_id'] -> customers.['id'], ['product_id'] -> products.['id'].

PRODUCTS:
Table 'products' has columns: id (INTEGER), name (VARCHAR(100)), category (VARCHAR(50)), price (NUMERIC(10, 2)), stock_quantity (INTEGER), .

✓ Schema inspection complete!
You now have 500 customers, 500 products, and 500 orders


### 3.2 Handle PostgreSQL-Specific Features

In [5]:
# PostgreSQL-specific query examples
postgres_examples = """
PostgreSQL-Specific Features to Consider:

1. **JSON/JSONB Columns:**
   SELECT data->>'name' as name FROM users;
   
2. **Array Columns:**
   SELECT * FROM products WHERE 'electronics' = ANY(categories);
   
3. **Full-Text Search:**
   SELECT * FROM articles 
   WHERE to_tsvector('english', content) @@ to_tsquery('postgresql');
   
4. **Window Functions:**
   SELECT name, salary, 
          RANK() OVER (PARTITION BY department ORDER BY salary DESC)
   FROM employees;
   
5. **CTEs (Common Table Expressions):**
   WITH top_customers AS (
       SELECT customer_id, SUM(amount) as total
       FROM orders GROUP BY customer_id
       ORDER BY total DESC LIMIT 10
   )
   SELECT * FROM top_customers;

6. **RETURNING Clause:** (Not useful for read-only)
   INSERT INTO users (name) VALUES ('Alice') RETURNING id;
"""

print(postgres_examples)


PostgreSQL-Specific Features to Consider:

1. **JSON/JSONB Columns:**
   SELECT data->>'name' as name FROM users;

2. **Array Columns:**
   SELECT * FROM products WHERE 'electronics' = ANY(categories);

3. **Full-Text Search:**
   SELECT * FROM articles 
   WHERE to_tsvector('english', content) @@ to_tsquery('postgresql');

4. **Window Functions:**
   SELECT name, salary, 
          RANK() OVER (PARTITION BY department ORDER BY salary DESC)
   FROM employees;

5. **CTEs (Common Table Expressions):**
   WITH top_customers AS (
       SELECT customer_id, SUM(amount) as total
       FROM orders GROUP BY customer_id
       ORDER BY total DESC LIMIT 10
   )
   SELECT * FROM top_customers;

6. **RETURNING Clause:** (Not useful for read-only)
   INSERT INTO users (name) VALUES ('Alice') RETURNING id;



## Section 4: Text-to-SQL with PostgreSQL

### 4.1 Initialize LLM and Query Engine

In [6]:
# Initialize LLM
llm = OpenAI(
    temperature=0.1,
    model="gpt-4o-mini",
    api_key=OPENAI_API_KEY
)

print("✓ LLM initialized")
print("Ready to create query engine once database is connected")

✓ LLM initialized
Ready to create query engine once database is connected


### 4.2 Create Query Engine (Example)

In [7]:
# Create query engine for your Supabase data
print("Initializing text-to-SQL query engine...")

# Option 1: Simple Query Engine (recommended for getting started)
query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["customers", "products", "orders"],
    llm=llm,
    verbose=True
)

print("✓ Query engine initialized!")
print("\n" + "="*60)
print("TRY THESE NATURAL LANGUAGE QUERIES:")
print("="*60)
print('  - "How many customers are there?"')
print('  - "What are the top 10 most expensive products?"')
print('  - "Show me customers from New York"')
print('  - "Which product category has the most items?"')
print('  - "What is the total revenue from all orders?"')
print('  - "List products with stock less than 50"')
print('  - "Which customer placed the most orders?"')
print("="*60)

# Example query
print("\n📝 Example Query:\n")
response = query_engine.query("How many customers are in the database?")
print(f"\n💬 Answer: {response}")

# Try another query
print("\n" + "="*60)
print("\n📝 Another Example:\n")
response = query_engine.query("What are the top 5 most expensive products?")
print(f"\n💬 Answer: {response}")

print("\n" + "="*60)
print("✓ Ready to answer your questions about the data!")
print("Run your own queries using: query_engine.query('your question here')")
print("="*60)

Initializing text-to-SQL query engine...
✓ Query engine initialized!

TRY THESE NATURAL LANGUAGE QUERIES:
  - "How many customers are there?"
  - "What are the top 10 most expensive products?"
  - "Show me customers from New York"
  - "Which product category has the most items?"
  - "What is the total revenue from all orders?"
  - "List products with stock less than 50"
  - "Which customer placed the most orders?"

📝 Example Query:



> Table Info: Table 'customers' has columns: id (INTEGER), name (VARCHAR(100)), email (VARCHAR(100)), city (VARCHAR(50)), created_at (TIMESTAMP), .
> Table Info: Table 'products' has columns: id (INTEGER), name (VARCHAR(100)), category (VARCHAR(50)), price (NUMERIC(10, 2)), stock_quantity (INTEGER), .


INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'customers' has columns: id (INTEGER), name (VARCHAR(100)), email (VARCHAR(100)), city (VARCHAR(50)), created_at (TIMESTAMP), .

Table 'products' has columns: id (INTEGER), name (VARCHAR(100)), category (VARCHAR(50)), price (NUMERIC(10, 2)), stock_quantity (INTEGER), .

Table 'orders' has columns: id (INTEGER), customer_id (INTEGER), product_id (INTEGER), quantity (INTEGER), total_amount (NUMERIC(10, 2)), order_date (TIMESTAMP),  and foreign keys: ['customer_id'] -> customers.['id'], ['product_id'] -> products.['id'].


> Table Info: Table 'orders' has columns: id (INTEGER), customer_id (INTEGER), product_id (INTEGER), quantity (INTEGER), total_amount (NUMERIC(10, 2)), order_date (TIMESTAMP),  and foreign keys: ['customer_id'] -> customers.['id'], ['product_id'] -> products.['id'].
> Table desc str: Table 'customers' has columns: id (INTEGER), name (VARCHAR(100)), email (VARCHAR(100)), city (VARCHAR(50)), created_at (TIMESTAMP), .

Table 'products' has columns: id (INTEGER), name (VARCHAR(100)), category (VARCHAR(50)), price (NUMERIC(10, 2)), stock_quantity (INTEGER), .

Table 'orders' has columns: id (INTEGER), customer_id (INTEGER), product_id (INTEGER), quantity (INTEGER), total_amount (NUMERIC(10, 2)), order_date (TIMESTAMP),  and foreign keys: ['customer_id'] -> customers.['id'], ['product_id'] -> products.['id'].


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...


💬 Answer: There are 500 customers in the database.


📝 Another Example:

> Table Info: Table 'customers' has columns: id (INTEGER), name (VARCHAR(100)), email (VARCHAR(100)), city (VARCHAR(50)), created_at (TIMESTAMP), .
> Table Info: Table 'products' has columns: id (INTEGER), name (VARCHAR(100)), category (VARCHAR(50)), price (NUMERIC(10, 2)), stock_quantity (INTEGER), .
> Table Info: Table 'orders' has columns: id (INTEGER), customer_id (INTEGER), product_id (INTEGER), quantity (INTEGER), total_amount (NUMERIC(10, 2)), order_date (TIMESTAMP),  and foreign keys: ['customer_id'] -> customers.['id'], ['product_id'] -> products.['id'].
> Table desc str: Table 'customers' has columns: id (INTEGER), name (VARCHAR(100)), email (VARCHAR(100)), city (VARCHAR(50)), created_at (TIMESTAMP), .

Table 'products' has columns: id (INTEGER), name (VARCHAR(100)), category (VARCHAR(50)), price (NUMERIC(10, 2)), stock_quantity (INTEGER), .

Table 'orders' has columns: id (INTEGER), customer_id (INTEGE

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


> Predicted SQL query: SELECT products.name, products.price FROM products ORDER BY products.price DESC LIMIT 5;


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



💬 Answer: The top 5 most expensive products are:

1. Tablet #332 - $2990.17
2. Smartphone Elite #14 - $2943.73
3. Headphones Portable 2024 #80 - $2939.59
4. Desktop Plus #364 - $2879.76
5. Monitor Elite Max #253 - $2879.60

✓ Ready to answer your questions about the data!
Run your own queries using: query_engine.query('your question here')
